In [ ]:

# Import libaries
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit.qpy as qpy

In [ ]:
# Contructs and runs BV circuit using provided oracle circuit and returns "secret number"
def bv_decode(oracle_circ: QuantumCircuit) -> int:
    
    # Convert the loaded oracle circuit to an instruction.
    oracle_inst = oracle_circ.to_instruction()
    
    # The oracle was built with (n+1) qubits (n input qubits, plus one ancilla).
    total_qubits = oracle_inst.num_qubits  # n + 1
    n = total_qubits - 1  # number of input qubits

    # Build the Bernstein-Vazirani circuit.
    circuit = QuantumCircuit(n + 1, n)

    # 1. Apply Hadamard gates to all input (query) qubits.
    for i in range(n):
        circuit.h(i)
    
    # 2. Prepare the ancilla in the |-> state.
    circuit.x(n)
    circuit.h(n)
    circuit.barrier()
    
    # 3. Append the oracle.
    circuit.append(oracle_inst, list(range(total_qubits)), list(range(n)))
    circuit.barrier()
    
    # 4. Apply Hadamard gates again to all input qubits.
    for i in range(n):
        circuit.h(i)
    circuit.barrier()
    
    # 5. Measure all input qubits.
    for i in range(n):
        circuit.measure(i, i)

    # Use QiskitRuntimeService to run the circuit on a real IBM Quantum backend.
    service = QiskitRuntimeService()
    # Choose a backend (for example, the least busy non-simulator device).
    backend = service.least_busy(operational=True, simulator=False)
    print("Using backend:", backend.name)
    
    # Optimize the circuit using a preset pass manager.
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    optimized_circuit = pm.run(circuit)
    

    sampler = Sampler(backend)
    job = sampler.run([optimized_circuit])
    
    job_result = job.result()
    pub_result = job_result[0]

    print("Job ID:", job.job_id())
    result = job.result()
    
    # Extract the measurement results.
    counts = pub_result.data.c.get_counts()

    secret_bitstring = max(counts, key=counts.get)
    recovered_secret = int(secret_bitstring, 2)
    return recovered_secret

In [ ]:
# 
def main():
    # Load the oracle circuit from the QPY file.
    qpy_filename = "oracle.qpy"
    with open(qpy_filename, "rb") as f:
        loaded_oracles = qpy.load(f)
    # qpy.load returns a list; take the first (and only) element.
    oracle_circ = loaded_oracles[0]
    
    # Run the BV algorithm to decode the secret using the loaded oracle.
    recovered_secret = bv_decode(oracle_circ)
    print(f"Recovered secret: {recovered_secret}")

if __name__ == "__main__":
    main()
